In [1]:
import os
os.chdir(r'C:\Users\guthu\Desktop\CHATBOT')
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import tensorflow_text as text

In [2]:
f = open('data.json')

In [3]:
data = json.load(f)

In [82]:
labels = []
patterns = []
responses = {}
for intent in data['intents']:
    for sentence in intent['patterns']:
        labels.append(intent['tag'].lower())
        patterns.append(sentence)
        responses[intent['tag'].lower()]=intent['responses']  

In [83]:
responses

{'greeting': ['Hello, thanks for visiting',
  'Good to see you again',
  'Hi there, how can I help?'],
 'goodbye': ['See you later, thanks for visiting',
  'Have a nice day',
  'Bye! Come back again soon.'],
 'thanks': ['Happy to help!', 'Any time!', 'My pleasure'],
 'hours': ["We're open every day from 9AM to 9PM",
  'Our working hours are 9AM to 9PM every day'],
 'location': ['We are on BH-5, LPU',
  'Our company is situated in BH-5, LPU',
  'We work from BH-5 LPU',
  'Our location is BH-5 LPU'],
 'payments': ['We accept VISA, Mastercard and AMEX',
  'We accept most major credit cards'],
 'services': ['We provide Web Penetration Testing,Android Penetration Testing,Docker Penetration Testing,Vulnerability Assessment,Cyber Crime investigation and many more services.']}

In [5]:
len(labels)

30

In [6]:
Classes = sorted(list(set(labels)))
len(Classes)

7

In [7]:
Labels=[]  
for label in Classes:
 bag = [] 
 for sentence in labels:
  T = [sentence.lower()]
  if label in T:
      bag.append(1)
  else:
      bag.append(0)
 Labels.append(bag)
Labels = np.transpose(np.array(Labels))

In [8]:
Labels.shape

(30, 7)

In [11]:
import nltk
from nltk.stem import WordNetLemmatizer
import re
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
def text_preprocessing(responses):
    lemmatizer = WordNetLemmatizer()
    S=[]
    W=[]
    for sentence in responses:
        x = re.sub('[^a-zA-Z]',' ',sentence)
        x = x.lower()
        words = word_tokenize(x)
        L=[]
        for word in words:
         x = word                  #lemmatizer.lemmatize(word)
         W.append(x)
         L.append(x)
        x = ' '.join(L)
        S.append(x)
        W = sorted(list(set(W)))
    return (S,W)

In [12]:
processed_text,words = text_preprocessing(patterns)

In [13]:
print(len(processed_text))

30


In [17]:
def onehot(patterns,words):  
  text_num=[]  
  for sentence in patterns:
    D = []
    for i in range(len(words)):
        D.append(0)    
        H=D
    T = sentence.lower()
    T = word_tokenize(sentence)
    for x in T:
        for j in range(len(H)):
            if words[j]==x:  
                H[j]=1
    text_num.append(H)
  text_num = np.array(text_num)
  return text_num

In [18]:
text_num = onehot(processed_text,words)

In [73]:
text_num

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [21]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(text_num,Labels,test_size=0.3,random_state=5)

In [22]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(21, 51)
(9, 51)
(21, 7)
(9, 7)


In [23]:
import tensorflow as tf
import tensorflow_text as text
from keras.models import Sequential
from keras.layers import Dense, Dropout,Embedding
from keras.optimizers  import Adam,SGD
from tensorflow.keras.models import load_model

In [24]:
from tensorflow.keras import layers
import keras_tuner as kt
import keras
from keras_tuner.tuners import RandomSearch
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(X_train.shape[1],)))
    for i in range(hp.Int('num_layers', 2,10)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(Y_train.shape[1], activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['AUC'])
    return model

In [25]:
tuner = RandomSearch(
    build_model,
    kt.Objective("val_auc", direction="max"),
    max_trials=50,
    executions_per_trial=1,
    directory=r'C:\Users\guthu\Desktop\CHATBOT\hyperparameters',
    project_name='Chatbot')

tuner.search(X_train,Y_train, epochs=300,batch_size=512, validation_data=(X_test, Y_test))

Trial 50 Complete [00h 00m 09s]
val_auc: 0.5751029253005981

Best val_auc So Far: 0.9526748657226562
Total elapsed time: 00h 12m 38s
INFO:tensorflow:Oracle triggered exit


In [47]:
models = tuner.get_best_models(num_models=1)
best_model = models[0]

In [48]:
best_model.build(input_shape=(X_train.shape[1],))
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 51)                0         
                                                                 
 dense (Dense)               (None, 128)               6656      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 96)                24672     
                                                                 
 dense_3 (Dense)             (None, 128)               12416     
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 192)               1

In [65]:
best_model.save('chatbot.h5')

In [66]:
with open('Classes.pkl','wb') as h:
    pickle.dump(Classes,h)

In [72]:
with open('words.pkl','wb') as h:
    pickle.dump(words,h)

In [84]:
with open('responses.pkl','wb') as h:
    pickle.dump(responses,h)

In [61]:
def model_predict(X,model,Classes):  
    p = model.predict(X)
    C=[]
    for i in range(len(p)):
       index = np.argmax(p[i])
       c = Classes[index]
       C.append(c)
    return C
from numpy.linalg import norm
def cosine_similarity(A,B):
    x = norm(A, axis=1).reshape(-1,1)
    cosine = np.dot(A,B)/(x*norm(B))
    return cosine

In [70]:
model_predict(X_test[:1],best_model,Classes)

1/1 [==============================] - 0s 24ms/step


['hours']

In [68]:
C=[]
for i in range(len(Y_test)):
   index = np.argmax(Y_test[i])
   c = Classes[index]
   C.append(c)
print(C)

['hours', 'services', 'payments', 'payments', 'location', 'thanks', 'greeting', 'hours', 'services']
